Discovery:

Split the calendar month by month.
The first column must be the names of the employees
The header must be the days of the specific month.
- Night employees: One week 2 nights, one week 5 days organized by:
2 days of work + 2 free days +  days of work. 2 people each night.
- Rest of employees: 3 people each morning, 2 people each evening.
At least one free day per week.



In [1]:
from datetime import date, timedelta
import holidays
import pandas as pd
import csv
import random

In [2]:
def calendar(year,country='ES',subdiv=None):
    """
    returns a dataframe with three columns:
    dates: with format [YYYY-MM-DD], ascending
    weekend: with value 1 if it's Saturday or Sunday, otherwise it returns 0.
    holidays: with value 1 if it's public holidays, otherwise it returns 0.
    """
    calendar=pd.DataFrame()

    def daterange(year):
        calendar = pd.DataFrame()
        calendar['Dates']=pd.date_range(f'{year}-01-01', f'{year}-12-31', freq='D')
        return calendar

    calendario = daterange(year)

    def weekends(calendar):
        calendar['weekday']= calendar['Dates'].apply(lambda x: x.weekday())
        weekend = {0:0,1:0,2:0,3:0,4:0,5:1,6:1}
        calendar['Weekend'] = calendar['weekday'].map(weekend)
        calendar.drop(columns='weekday',inplace=True)
        return calendar

    calendario = weekends(calendario)

    def public_holidays(calendar):
        es_cl_holidays = holidays.country_holidays(country,subdiv)
        calendar['Holidays']=calendar['Dates'].apply(lambda x: x in es_cl_holidays).map(int)
        return calendar
    calendario = public_holidays(calendario)

    def times(calendario):
        times=['Morning','Evening','Night']
        for i in times:
            calendario[i] = pd.Series(0)
        return calendario
    calendario = times(calendario)
    calendario['Month'] = calendario['Dates'].apply(lambda x: x.strftime('%b'))
    calendario['Week Number'] = calendario['Dates'].dt.isocalendar().week
    return calendario



In [3]:
calendario = calendar(2023,'ES','CL')
calendario['Holidays'].value_counts()

0    353
1     12
Name: Holidays, dtype: int64

In [4]:
#calendario['Week Number'] = calendario['Dates'].dt.isocalendar().week
calendario

,Dates,Weekend,Holidays,Morning,Evening,Night,Month,Week Number
0,2023-01-01,1,0,0.0,0.0,0.0,Jan,52
1,2023-01-02,0,1,NaN,NaN,NaN,Jan,1
2,2023-01-03,0,0,NaN,NaN,NaN,Jan,1
3,2023-01-04,0,0,NaN,NaN,NaN,Jan,1
4,2023-01-05,0,0,NaN,NaN,NaN,Jan,1
...,...,...,...,...,...,...,...,...
360,2023-12-27,0,0,NaN,NaN,NaN,Dec,52
361,2023-12-28,0,0,NaN,NaN,NaN,Dec,52
362,2023-12-29,0,0,NaN,NaN,NaN,Dec,52
363,2023-12-30,1,0,NaN,NaN,NaN,Dec,52


In [5]:
calendario.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 365 entries, 0 to 364
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Dates        365 non-null    datetime64[ns]
 1   Weekend      365 non-null    int64         
 2   Holidays     365 non-null    int64         
 3   Morning      1 non-null      float64       
 4   Evening      1 non-null      float64       
 5   Night        1 non-null      float64       
 6   Month        365 non-null    object        
 7   Week Number  365 non-null    UInt32        
dtypes: UInt32(1), datetime64[ns](1), float64(3), int64(2), object(1)
memory usage: 21.9+ KB


In [6]:
def number_workers():
    worker= input("how many workers do you have? >>")
    try:
        worker = int(worker)
        return worker
    except ValueError:
        print("That's not a number!")

# Employees Dataframe

def df_workers(number_of_workers):

    def name_workers():
        name = input('How is your worker called >>')
        if name.strip().isdigit():
            print("That's not a name!")
        else:
            name=name.capitalize()
            return name

    def age_workers():
        age= input("How old is your worker? >>")
        try:
            age = int(age)
            return age
        except ValueError:
            print("That's not a number!")

    def experience_workers():
        experience= input("How many years of experience this person has? >>")
        try:
            experience = int(experience)
            return experience
        except ValueError:
            print("That's not a number!")

    def salary_workers():
        salary = input("What's the annual salary of your worker? (In €) >>")
        try:
            salary = float(salary)
            return salary
        except ValueError:
            print("That's not a number!")

    def times_workers():
        timer =[1,2,3,4,5,6]
        time = input("When does your worker works? >>\n"
                "1 for Morning and Evening\n"
                "2 for Night\n")
        try:
            time = int(time)
            if time in timer:
                return time
            else: print('Select a correct value')
        except ValueError:
            print("That's not a number!")

    def gender_workers():
        genders ={1:'Female',2:'Male',3:'Non-binary'}
        gender = input("What's the gender of your worker? >>\n"
                    "1 for Female\n"
                    "2 for Male\n"
                    "3 for Non-binary\n")
        try:
            gender = int(gender)
            if gender in genders:
                return genders[gender]
            else: print('Select a correct value')
        except ValueError:
            print("That's not a number!")

    def weekend_workers():
        weekends ={1:'Yes',2:'No'}
        weekend = input("Does your worker work during weekends? >>\n"
                    "1 for No\n"
                    "2 for Yes\n")
        try:
            weekend = int(weekend)
            if weekend in weekends:
                return weekend-1
            else: print('Select a correct value')
        except ValueError:
            print("That's not a number!")

    def holiday_workers():
        holidays ={1:'Yes',2:'No'}
        holiday = input("Does your worker work during public holidays? >>\n"
                    "1 for No\n"
                    "2 for Yes\n")
        try:
            holiday = int(holiday)
            if holiday in holidays:
                return holiday - 1
            else: print('Select a correct value')
        except ValueError:
            print("That's not a number!")
    def percentage_work():
        percentage = input("What's is the percentage your employee works? >>\n"
                    "Add it in decimals, for example:\n"
                    "20% = 0.2\n"
                    "65% = 0.65\n")
        try:
            percentages = float(percentage)
            return percentages
        except ValueError:
            print("That's not a number!")


    def data_workers(number_of_workers):
        data={}
        for i in range(number_of_workers):
            name = name_workers()
            age = age_workers()
            experience = experience_workers()
            salary = salary_workers()
            schedule = times_workers()
            gender = gender_workers()
            weekend = weekend_workers()
            holiday = holiday_workers()
            percentage = percentage_work()
            data[i] = {'name': name,'age': age,'experience':experience,'salary':salary,'schedule':schedule,'gender':gender,'weekend':weekend,'holiday':holiday,'percentage':percentage}
        return data
    df = data_workers(number_of_workers)
    df = pd.DataFrame.from_dict(df,orient='index')
    return df

In [4]:
data=pd.read_csv('/Users/monic/code/monicasainer/calendar/raw_data/Project.csv',delimiter=';',header=0,index_col=False)

In [5]:
data.columns

Index(['name', 'age', 'experience', 'salary', 'schedule', 'gender', 'weekend',
       'holiday', 'percentage'],
      dtype='object')

In [9]:
def accounting(workers_df,calendar_df):
    workers=list(workers_df['name'])
    for i in workers:
        calendar_df[f'{i} T.C.']=0
        calendar_df[f'{i} W.C.']=0
        calendar_df[f'{i} M.C.']=0
    return calendar_df

In [10]:
calendario = accounting(data,calendario)

In [ ]:
calendario

In [11]:
def nights_assignment(workers_df,calendar_df):
    def assignment():
        worker_nights=workers_df[workers_df['schedule']==2]['name']
        total_hours={}
        total_days = 0
        for x in worker_nights:
            percentage = float(workers_df[workers_df['name']==x]['percentage'])
            initial_days=round(len(calendar_df['Night'])/len(worker_nights))
            total_hours[x]=round(initial_days*percentage)
            total_days +=total_hours[x]
        diff_days=len(calendar_df['Night'])-total_days
        while (diff_days >= len(worker_nights)) & (total_days >= len(calendar_df['Night'])) :
            for i in worker_nights:
                percentage = float(data[data['name']==i]['percentage'])
                initial_days=round(diff_days/len(worker_nights))
                total_hours[i]+=round(initial_days*percentage)
                total_days += round(initial_days*percentage)
            diff_days=len(calendar_df['Night'])-total_days
        quotient,remainder = divmod(diff_days,len(worker_nights))
        for t in worker_nights:
            total_hours[t]+=quotient
            total_days += quotient
        names = random.sample(list(worker_nights),remainder)
        for p in names:
            total_hours[p]+=1
        return total_hours
    
    def setting_nights():
        total_hours =assignment()
        worker_nights=workers_df[workers_df['schedule']==2]['name']
        indexes = list(range(len(calendar_df['Night'])))
        random.shuffle(indexes)

        for i in worker_nights:
            index = indexes[:total_hours[i]]
            indexes = indexes[total_hours[i]:]
            for x in index:
                calendar_df['Night'].iloc[x]= i
        return calendar_df
    calendar_df = setting_nights()
    
    def summary():
        worker_nights=workers_df[workers_df['schedule']==2]['name']
        for x in worker_nights:
            count = 0
            for i in range(len(calendar_df.index)):
                if (calendar_df['Night'].iloc[i] == x):
                    count += 1
                    calendar_df[f'{x} T.C.'][i] = count
    return calendar_df

In [ ]:
data['percentage'].iloc[11]

In [ ]:
(data['percentage']==1.0) & (data['schedule']==2)


In [ ]:
worker_nights=data[data['schedule']==2]['name']
total_hours={}
total_days = 0
for x in worker_nights:
    percentage = float(data[data['name']==x]['percentage'])
    initial_days=round(len(calendario['Night'])/len(worker_nights))
    total_hours[x]=round(initial_days*percentage)
    total_days +=total_hours[x]
diff_days=len(calendario['Night'])-total_days
while (diff_days >= len(worker_nights)) & (total_days >= len(calendario['Night'])) :
    for i in worker_nights:
        percentage = float(data[data['name']==i]['percentage'])
        initial_days=round(diff_days/len(worker_nights))
        total_hours[i]+=round(initial_days*percentage)
        total_days += round(initial_days*percentage)
    diff_days=len(calendario['Night'])-total_days
quotient,remainder = divmod(diff_days,len(worker_nights))
for t in worker_nights:
    total_hours[t]+=quotient
    total_days += quotient
names = random.sample(list(worker_nights),remainder)
for p in names:
    total_hours[p]+=1
total_hours

In [ ]:
worker_nights=data[data['schedule']==2]['name']
indexes = list(range(len(calendario['Night'])))
random.shuffle(indexes)

for i in worker_nights:
    index = indexes[:total_hours[i]]
    indexes = indexes[total_hours[i]:]
    for x in index:
        calendario['Night'].iloc[x]= i
calendario

In [12]:
calendario = nights_assignment(data,calendario)

/var/folders/bs/pzl9llpn4z9d1kxlsk4k87y00000gn/T/ipykernel_35770/345803320.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  calendar_df['Night'].iloc[x]= i


In [13]:
calendario['Night'].value_counts()

Paco       103
Mariana    102
Lucia      102
Carlos      58
Name: Night, dtype: int64

In [ ]:
def holiday_assignment(workers_df,calendar_df):
    def split(a, n):
        k, m = divmod(len(a), n)
        return (a[i*k+min(i, m):(i+1)*k+min(i+1, m)] for i in range(n))
    worker_holidays = list(workers_df[workers_df['holiday']==1]['name'])*(len(workers_df[workers_df['holiday']==1]['name'])-1)
    public_holidays=list(calendar_df[calendar_df['Holidays']==1].index)
    random.shuffle(public_holidays)
    split_index = list(split(public_holidays,len(list(workers_df[workers_df['holiday']==1]['name']))))
    for i in range(len(split_index)):
        indexes = split_index[i]
        names_selected = random.sample(worker_holidays,2)
        while names_selected[0]==names_selected[1]:
            names_selected = random.sample(worker_holidays,2)
        for x in indexes:
            calendar_df['Morning'].iloc[x]=names_selected[0]
            calendar_df['Evening'].iloc[x]=names_selected[1]
        worker_holidays.remove(names_selected[0])
        worker_holidays.remove(names_selected[1])
    workers=list(workers_df[workers_df['holiday']==1]['name'])
    for x in workers:
        count = 0
        for i in range(len(calendar_df.index)):
            if (calendar_df['Morning'].iloc[i] == x or calendar_df['Evening'].iloc[i]== x or calendar_df['Night'].iloc[i] == x):
                count += 1
                calendar_df[f'{x} T.C.'][i] = count
    return calendar_df

In [ ]:
calendario = holiday_assignment(data,calendario)

In [ ]:
calendario['Morning'].value_counts()

In [ ]:
calendario['Evening'].value_counts()

In [ ]:
calendario['Carlos T.C.'].value_counts()

In [ ]:
def night_assignment(workers_df,calendar_df):
    def split(a, n):
        k, m = divmod(len(a), n)
        return (a[i*k+min(i, m):(i+1)*k+min(i+1, m)] for i in range(n))
    worker_holidays = list(workers_df[workers_df['schedule']==2]['name'])*(len(workers_df[workers_df['holiday']==1]['name'])-1)
    public_holidays=list(calendar_df[calendar_df['Holidays']==1].index)
    random.shuffle(public_holidays)
    split_index = list(split(public_holidays,len(list(workers_df[workers_df['holiday']==1]['name']))))
    for i in range(len(split_index)):
        indexes = split_index[i]
        names_selected = random.sample(worker_holidays,2)
        while names_selected[0]==names_selected[1]:
            names_selected = random.sample(worker_holidays,2)
        for x in indexes:
            calendar_df['Morning'].iloc[x]=names_selected[0]
            calendar_df['Evening'].iloc[x]=names_selected[1]
        worker_holidays.remove(names_selected[0])
        worker_holidays.remove(names_selected[1])
    workers=list(workers_df[workers_df['holiday']==1]['name'])
    for x in workers:
        count = 0
        for i in range(len(calendar_df.index)):
            if (calendar_df['Morning'].iloc[i] == x or calendar_df['Evening'].iloc[i]== x or calendar_df['Night'].iloc[i] == x):
                count += 1
                calendar_df[f'{x} T.C.'][i] = count
    return calendar_df